# Make plots map
Create some plots on a map.

In [7]:
using PyPlot
using NCDatasets
using PyCall
using Dates
using Glob
const plt = PyPlot
ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
include("./config.jl")
include("./ME4OH.jl")
mainproj = ccrs.Mollweide(central_longitude=200)
datacrs = ccrs.PlateCarree();
coast = cfeature.GSHHSFeature(scale="h")

[ Info: File already downloaded


PyObject <cartopy.feature.GSHHSFeature object at 0x795cdb3429b0>

In [8]:
@info("Figures will be saved in $(figdir)")

[ Info: Figures will be saved in ../figures/


## Observations 

In [9]:
thetimeperiod = timeperiod1
datadirdisk = joinpath(databasedir, "$(thetimeperiod[1])-$(thetimeperiod[end])")
isdir(datadirdisk) ? @info("Data directory exists") : @error("Data directory does't exist")

[ Info: Data directory exists


In [10]:
datafilelist = ME4OH.get_filelist(datadirdisk)
datafile = datafilelist[1]

[ Info: Found 432 files


"/media/ctroupin/T7 Shield/000060826v009/data/en4.1.1/1979-2014/ofam3-jra55.all.EN.4.1.1.f.profiles.g10.197901.update.extra.danom.197901_201412.nc"

In [63]:
for indexlayer = 1:3
    lon, lat, dates, dohc, adohc, dadohc, dohc_mask, bounds, depth_level_thickness = ME4OH.read_profile(datafile);
    goodvalues = goodvalues = .!(isnan.(dohc[indexlayer,:]));
    
    fig = plt.figure()
    ax = plt.subplot(111, projection=mainproj)
    ax.set_global()
    scat = ax.scatter(lon, lat, s=3, c=dohc[indexlayer,:], transform=datacrs)
    cb = plt.colorbar(scat, shrink=.6)
    cb.set_label("TJ/m²", rotation=0, ha="left")
    ax.set_title("Density of Ocean Heat Content\n $(Dates.monthname(Dates.month(dates[1]))) $(Dates.year(dates[1])), layer [$(depthlayers[indexlayer][1])-$(depthlayers[indexlayer][2])] m")
    ax.add_feature(coast, lw=.5, zorder=4)
    fname = "dohc_$(Dates.year(dates[1]))$(lpad(Dates.month(dates[1]), 2, '0'))_layer$(indexlayer)"
    plt.savefig(joinpath(figdir, fname))
    plt.close()
end    

## Gridded fields

In [71]:
outputfile = joinpath(mainoutputdir, "experiment-A/dohc/OHC_1979_2014_lev0_286.6_expA_DIVAnd.nc")

"../output/experiment-A/dohc/OHC_1979_2014_lev0_286.6_expA_DIVAnd.nc"

In [72]:
nc = NCDataset(outputfile, "r")
dates = nc["time"][:]
dohc = nc["dohc"][:,:,:]
close(nc)

closed Dataset

In [75]:
timeindex = findfirst(dates .== DateTime(2005, 1, 1))

313

In [82]:
fig = plt.figure()
ax = plt.subplot(111, projection=mainproj)
ax.set_global()
pcm = ax.pcolormesh(longrid, latgrid, dohc[:,:,timeindex]', transform=datacrs)
cb = plt.colorbar(pcm, shrink=.6)
cb.set_label("TJ/m²", rotation=0, ha="left")
ax.set_title("Interpolated density of Ocean Heat Content\nExperiment A\n $(Dates.monthname(Dates.month(dates[1]))) $(Dates.year(dates[1])), layer [$(depthlayers[indexlayer][1])-$(depthlayers[indexlayer][2])] m")
ax.add_feature(coast, lw=.5, zorder=4)
fname = "dohc_expA_$(Dates.year(dates[1]))$(lpad(Dates.month(dates[1]), 2, '0'))_layer$(indexlayer)"
plt.savefig(joinpath(figdir, fname))
plt.close()
plt.show() 

## Temperature observations

In [37]:
timeperiodtext = "$(thetimeperiod[1])01_$(thetimeperiod[end])12"
fnameprefix = "ofam3-jra55.all.EN.4.1.1.f.profiles.g10"
fnamesuffix = "update.extra.danom"

for mm = 1:12
    @info("Working on month $(mm)")
    datafilelistmonth = glob("$(fnameprefix).*$(lpad(mm, 2, "0")).$(fnamesuffix).$(timeperiodtext).nc", datadirdisk)
    @info("Found $(length(datafilelistmonth)) files")

    obslon, obslat, obsdates, obsdepth, T, S = ME4OH.read_TS_data(datafilelistmonth);
    nobs = length(obslon)
    @info("Total number of observations: $(nobs)")
    
    for thedepth in [obsdepth[1], obsdepth[10], obsdepth[20]]
    
        gooddepth = findall(obsdepth .== thedepth);
    
        fig = plt.figure()
        ax = plt.subplot(111, projection=mainproj)
        ax.set_global()
        scat = ax.scatter(obslon[gooddepth], obslat[gooddepth], s=1, c=T[gooddepth], cmap=plt.cm.RdYlBu_r,
        transform=datacrs)
        cb = plt.colorbar(scat, shrink=.6)
        cb.set_label("°C", rotation=0, ha="left")
        ax.set_title("Temperature observations at $(round(thedepth, digits=1)) m in $(Dates.monthname(mm))\nfor the time period $(thetimeperiod[1]) - $(thetimeperiod[end])")
        ax.add_feature(coast, lw=.5, zorder=4)
        fname = "temperature_month$(lpad(mm, 2, '0'))_depth$(round(thedepth))_period$(thetimeperiod[1])_$(thetimeperiod[end]).jpg"
        plt.savefig(joinpath(figdir, fname))
        #plt.show()
        plt.close()
    end
    
end

## Temperature climatology

In [12]:
thetimeperiod = timeperiod1
climfile = "../output/experiment-C/temperature_climatology_$(thetimeperiod[1])_$(thetimeperiod[end]).nc"
isfile(climfile) ? @info("Working on file $(climfile)") : @error("File $(climfile) doesn't exist")

[ Info: Working on file ../output/experiment-C/temperature_climatology_1979_2014.nc


In [31]:
function plot_clim(longrid::StepRangeLen, latgrid::StepRangeLen, T::Matrix{Union{Missing, Float64}}, 
        thedepth::Float64, timeperiod::UnitRange{Int64}, mm::Int64, thedate::DateTime)
    fig = plt.figure()
    ax = plt.subplot(111, projection=mainproj)
    ax.set_global()
    pcm = ax.pcolormesh(longrid, latgrid, T', transform=datacrs, cmap=plt.cm.RdYlBu_r)
    cb = plt.colorbar(pcm, shrink=.6)
    cb.set_label("°C", rotation=0, ha="left")
    ax.set_title("Interpolated temperature at $(round(thedepth, digits=1)) m\nin $(Dates.monthname(Dates.month(thedate))) [$(thetimeperiod[1])-$(thetimeperiod[end])]")
    ax.add_feature(coast, lw=.5, zorder=4)
    fname = "temperature_clim_month$(lpad(mm, 2, '0'))_depth$(Int64(round(thedepth)))_period$(thetimeperiod[1])_$(thetimeperiod[end]).jpg"
    @info(fname)
    plt.savefig(joinpath(figdir, fname))
    plt.close()
    return nothing
end

plot_clim (generic function with 2 methods)

In [32]:
NCDataset(climfile, "r") do nc
    depth = nc["depth"][:]

    for mm = 1:1
        @info("Working on month $(mm)")

        for thedepth in [depth[1], depth[10], depth[20]]

            gooddepth = findfirst(depth .== thedepth);
            T = nc["temperature"][:,:,gooddepth,mm]
            thedate = nc["time"][mm]

            plot_clim(longrid, latgrid, T, thedepth, thetimeperiod, mm, thedate)

        end
    end

end

[ Info: Working on month 1
[ Info: temperature_clim_month01_depth2_period1979_2014.jpg
[ Info: temperature_clim_month01_depth57_period1979_2014.jpg
[ Info: temperature_clim_month01_depth155_period1979_2014.jpg


closed Dataset